# ***<u>librariess***

In [25]:
import numpy as np
import math
import random
import copy

# ***<u>Generate <u>Sudoku***

In [26]:
def generate_sudoku():
    
    """
        Function to generate a Sudoku grid

        
        Fill the diagonal boxes of the Sudoku grid
        Solve the Sudoku grid
        Remove elements from th  grid to create a puzzle
    """
    grid = [[0 for i in range(9)] for i in range(9)]
    fill_diagonal(grid)
    l = [0,9]
    i = l[0]
    j = l[1]
    fill_unassigned_locations(grid,i,j)
    remove_elements(grid,random.randint(25,55))
    return grid
    

def fill_diagonal(grid):
    
    """
        Function to fill the diagonal boxes of the Sudoku grid
    """
    for i in range(0, 9, 3):
        fill_box(grid, i, i)


def fill_box(grid, row, col):

    """
        Function to fill a box with random values
    """
    num = 0
    for i in range(3):
        for j in range(3):
            while True:
                num = math.floor(random.random() * 9 + 1)
                if not used_in_box(grid,row, col, num):
                    break
            grid[row + i][col + j] = num

def is_safe(grid, row, col, num):

    """
        Function to check if it is safe to place a number in a particular position
    """

    return (
        not used_in_row(grid, row, num)
        and not used_in_column(grid, col, num)
        and not used_in_box(grid, row - row % 3, col - col % 3, num)
    )


# 
def used_in_row(grid, row, num):

    """
        Function to check if a number is used in a row
    """
    for i in range(9):
        if grid[row][i]  == num:
            return True
    return False


def used_in_column(grid, col, num):

    """
        Function to check if a number is used in a column
    """
    for i in range(9):
        if grid[i][col]  == num:
            return True
    return False


def used_in_box(grid, box_start_row, box_start_col, num):

    """
        Function to check if a number is used in a 3x3 box
    """
    for i in range(3):
        for j in range(3):
            if grid[box_start_row + i][box_start_col + j] == num:
                return True
    return False

def find_unassigned_location(grid,l):

    """
        Function to find an unassigned location in the grid
    """
    for i in range(9):
        for j in range(9):
            if(grid[i][j]== 0):
                l[0]= i
                l[1]= j
                return True
    return False
         
def fill_unassigned_locations(grid,i,j):
    if i == 8 and j == 9:
        return True
    
    if j == 9:
        i += 1
        j = 0
    if grid[i][j] != 0:
        return fill_unassigned_locations(grid,i,j + 1)
    
    for num in range(1,10):
        if is_safe(grid,i,j,num):
            grid[i][j] = num
            if fill_unassigned_locations(grid,i,j + 1):
                return True
            grid[i][j] = 0
    return False
def remove_elements(grid, num_elements):
    
    """
        Function to remove elements from the grid
    """
    
 
    while (num_elements != 0):
        i = math.floor(random.random() * 9 + 1) - 1
        j = math.floor(random.random() * 9 + 1) - 1
        if (grid[i][j] != 0):
            num_elements -= 1
            grid[i][j] = 0
 
    return grid

# ***<u>BackTracking***

In [27]:
steps = 0
def solve_sudoku(grid):
    """
        Function to solve the Sudoku grid using backtracking
    """
    global steps
    l =[0, 0]
 
 # If there is no unassigned 
 # location, we are done  
    if(not find_unassigned_location(grid, l)):
        return True

    # Assigning list values to row and col 
    # that we got from the above Function 
    row = l[0]
    col = l[1]

    # consider digits 1 to 9
    for num in range(1, 10):
        steps += 1
     # if looks promising
        if(is_safe(grid,row, col, num)):
    
      # make tentative assignment
            grid[row][col]= num

      # return, if success, 
      # ya ! 
            if(solve_sudoku(grid)):
                return True

      # failure, unmake & try again
            grid[row][col] = 0
    
    # this triggers backtracking   
    return False
def display_grid(grid):

    """
        Function to display the Sudoku grid
    """
    print(f'Backtracking Solved in {steps} steps')
    for i in range(9):
        for j in range(9):
            print(grid[i][j], end=" ")
        print()
    

# ***<u>CSP***

In [28]:
def solve_sudoku_csp(grid):
    """
    Function to solve the Sudoku grid using Constraint Satisfaction Problem (CSP)
    """
    steps = 0
    def create_domains(grid):
        """
        Function to create domains for each cell in the grid
        """
        domains = {}
        for i in range(9):
            for j in range(9):
                domains[(i, j)] = []
                if grid[i][j] == 0:
                    for k in range(1,10):
                        if is_valid_assignment(i, j, k, grid):
                            domains[(i,j)].append(k)
                else:
                    domains[(i, j)] = grid[i][j]
        return domains

    def is_valid_assignment(i, j, val, assignment):
        """
        Function to check if assigning a value to a cell is valid
        Check if the value is already used in the same row
        Check if the value is already used in the same column
        Check if the value is already used in the same 3x3 box
        """
        
        if is_safe(grid,i,j,val):
            return True
        return False
        '''for row in range(9):
            if assignment[row][j] == val or assignment[i][row] == val:
                return False

        start_row, start_col = 3 * (i // 3), 3 * (j // 3)
        for row in range(start_row, start_row + 3):
            for col in range(start_col, start_col + 3):
                if assignment[row][col] == val:
                    return False

        return True'''

    def find_unassigned_location(assignment):
        """
        Function to find an unassigned location in the grid
        """
        for i in range(9):
            for j in range(9):
                if not isinstance(assignment[(i,j)], int):
                    return i, j
        return -1, -1

    def solve_csp(assignment):
        nonlocal domains
        nonlocal steps
        global final_grid
        
        i, j = find_unassigned_location(assignment)
        if i == -1 and j == -1:
            return True  
        assignment = domains
        for val in assignment[(i, j)]:
            steps += 1
            x = assignment[(i, j)]
            y = grid[i][j]
            assignment[(i,j)] = val
            domains[(i, j)] = val
            grid[i][j] = val
            domains = create_domains(grid)
            if solve_csp(assignment):
                final_grid = domains
                return True
            assignment[(i,j)] = x
            grid[i][j] = y
        return False
    domains = create_domains(grid)
    assignment = domains
    if solve_csp(assignment):
        grid = [[final_grid[(i, j)] for j in range(9)] for i in range(9)]
        print (f'CSP Solved Sudoku in {steps} steps')
        return grid
    return None

# ***<u>Show <u>Result***

In [29]:
# Function to initialize the Sudoku grid
def initializing_grid():
    print("\nInitial Sudoku\n")
    sudoku_grid = generate_sudoku()
    for i in sudoku_grid:
        for j in i:
            print(j, end=' ')
        print("")
    print(end='\n\n\n\n\n')
    return sudoku_grid


# Function to solve the Sudoku grid using backtracking
def backtracking_answer(sudoku_grid):
    print("Back Tracking Answer:\n\n")
    if solve_sudoku(sudoku_grid):
        print("Sudoku solved successfully:")
        display_grid(sudoku_grid)
    else:
        print("No solution exists for the given Sudoku.")
    print(end='\n\n\n\n\n')

# Function to solve the Sudoku grid using CSP
def csp_answer(sudoku_grid):
    print("CSP Answer:\n")
    solved_grid = solve_sudoku_csp(sudoku_grid)
    if solved_grid is not None:
        print("Sudoku solved successfully:")
        for row in solved_grid:
            for r1 in row:
                print(r1, end=' ')
            print()
    else:
        print("No solution exists for the given Sudoku.")
    print(end='\n\n\n\n\n')

In [30]:
# Generate and display the initial Sudoku grid
generate_sudoku = initializing_grid()
generate_sudoku1 = copy.deepcopy(generate_sudoku)
generate_sudoku2 = copy.deepcopy(generate_sudoku)

# Solve the Sudoku grid using backtracking
backtracking_answer(generate_sudoku1)
# Solve the Sudoku grid using CSP
csp_answer(generate_sudoku2)


Initial Sudoku

0 5 3 0 0 0 0 0 0 
8 7 2 1 0 4 5 6 0 
0 0 0 0 0 0 3 0 7 
0 2 5 0 0 8 0 0 3 
0 3 0 5 4 0 6 1 0 
6 4 0 0 0 0 2 9 0 
0 0 0 3 8 0 0 0 0 
0 9 0 4 0 7 0 0 6 
0 0 0 6 0 1 0 5 0 





Back Tracking Answer:


Sudoku solved successfully:
Backtracking Solved in 14953 steps
9 5 3 2 7 6 1 8 4 
8 7 2 1 3 4 5 6 9 
4 1 6 8 5 9 3 2 7 
1 2 5 9 6 8 4 7 3 
7 3 9 5 4 2 6 1 8 
6 4 8 7 1 3 2 9 5 
2 6 7 3 8 5 9 4 1 
5 9 1 4 2 7 8 3 6 
3 8 4 6 9 1 7 5 2 





CSP Answer:

CSP Solved Sudoku in 1682 steps
Sudoku solved successfully:
9 5 3 2 7 6 1 8 4 
8 7 2 1 3 4 5 6 9 
4 1 6 8 5 9 3 2 7 
1 2 5 9 6 8 4 7 3 
7 3 9 5 4 2 6 1 8 
6 4 8 7 1 3 2 9 5 
2 6 7 3 8 5 9 4 1 
5 9 1 4 2 7 8 3 6 
3 8 4 6 9 1 7 5 2 





